In [1]:
# !pip install openpyxl

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForLanguageModeling, \
    DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, T5Config
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from transformers.optimization import get_scheduler
from transformers import EarlyStoppingCallback
from torch.optim import AdamW



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained T5 model and tokenizer
model_name = "t5-small"  # or "t5-base" for better performance
# model_name = "t5-base"

# config = T5Config(num_layers=8, dropout_rate= 0.2)
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name, config=config).to(device)

# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
# # model.config.dropout_rate = 0.2

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.config.dropout_rate = 0.2

c:\Users\jites\anaconda3\envs\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# # print(len(tokenizer.vocab.keys()))

# # tokenizer.add_tokens(['<START_JSON>', '<END_JSON>'])
# special_tokens = {"additional_special_tokens": ['<START_JSON>', '<END_JSON>']}
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# # print(len(tokenizer.vocab.keys()))
# print(tokenizer.tokenize("<START_JSON> Hi, I am a {Robot} <END_JSON>"))


In [5]:
import json

with open(r'C:\Users\jites\Desktop\Project_folder\json_dataset\december21\data_10000.json', 'r') as f:
    data = json.load(f)

raw_dataset = Dataset.from_list(data)

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )



data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Use the same tokenizer as your model
    model=None,           # Specify model if you want model-specific padding/truncation
    padding=True,         # Dynamically pad sequences in the batch
    # max_length=512,       # Optional: Truncate to max length
    return_tensors="pt"   # Return PyTorch tensors (or 'tf' for TensorFlow)
)

In [6]:
# from torch.nn.utils.rnn import pad_sequence
# import torch

# def custom_data_collator(batch):
#     # Split inputs and labels
#     inputs = [item['input_ids'] for item in batch]
#     labels = [item['labels'] for item in batch]
    
#     # Pad sequences
#     inputs_padded = pad_sequence([torch.tensor(x) for x in inputs], batch_first=True, padding_value=tokenizer.pad_token_id)
#     labels_padded = pad_sequence([torch.tensor(x) for x in labels], batch_first=True, padding_value=-100)  # For ignoring padded tokens in loss
    
#     # Create attention masks
#     attention_mask = (inputs_padded != tokenizer.pad_token_id).long()
    
#     return {
#         "input_ids": inputs_padded,
#         "attention_mask": attention_mask,
#         "labels": labels_padded
#     }

# from torch.utils.data import DataLoader

# # Assuming `train_dataset` is your tokenized dataset
# train_dataloader = DataLoader(
#     train_dataset,
#     batch_size=16,
#     collate_fn=data_collator  # Use DataCollatorForSeq2Seq or your custom collator
# )


In [7]:
print(raw_dataset['input_text'][0])
print(raw_dataset['target_text'][0])

Issues such as acute liver failure and feeling of general weakness prompted tests like Colorectal Cancer Screening, Audiogram. Medicines include take it for 2 days strocit plus tab before food only afternoon and night . Follow up date is 30 days. Follow stress management techniques like meditation or yoga., Avoid self-medicating without consulting a doctor.
{'ChiefComplaint': 'The patient is experiencing acute liver failure, feeling of general weakness.', 'Symptoms': ['acute liver failure', 'feeling of general weakness'], 'LabTest': ['Colorectal Cancer Screening', 'Audiogram'], 'Advice': ['Follow stress management techniques like meditation or yoga.', 'Avoid self-medicating without consulting a doctor.'], 'FollowUp': 'Follow up date is 30 days.', 'Medicines': [{'name': 'strocit plus tab', 'quantity': '4', 'route': 'oral', 'instruction': 'before food', 'duration': '2 days', 'frequency': '0-1-1'}]}


In [8]:
# Preprocess the data
def preprocess_function(examples):
    inputs = tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=256)
    targets = tokenizer(examples['target_text'], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = targets['input_ids']
            ## OR
    # # Add decoder_input_ids (shift the target by 1 for causal generation)
    # inputs['decoder_input_ids'] = targets['input_ids']  # This can also be targets['input_ids'][shifted]
            ## OR
    #   # Add decoder input IDs and labels
    # inputs['decoder_input_ids'] = targets['input_ids']  # Optional for training
    # inputs['labels'] = [
    #     [(token if token != tokenizer.pad_token_id else -100) for token in target] 
    #     for target in targets['input_ids']
    # ]  # Mask out padding tokens in the labels
    return inputs

# Tokenize the dataset
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 10000/10000 [00:02<00:00, 3453.16 examples/s]


In [9]:
tokenized_dataset

Dataset({
    features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [10]:
print(tokenized_dataset['input_text'][5])
print(tokenized_dataset['target_text'][5])

The patient describes issues involving trouble in straight and mucoid sputum. and medicines involve Stamlo 5Mg Tablets afternoon and night 8 tablets  4 days , ZECAL 500MG TABLETS 24 tablets  12 days noon and night . Suggested tests include Semen Analysis, Luteinizing Hormone (LH), Holter Monitoring and suggested tests include Dobutamine Stress Echocardiogram.
{'ChiefComplaint': 'The patient is experiencing trouble in straight, mucoid sputum.', 'Symptoms': ['trouble in straight', 'mucoid sputum'], 'LabTest': ['Semen Analysis', 'Luteinizing Hormone (LH)', 'Holter Monitoring', 'Dobutamine Stress Echocardiogram'], 'Advice': [], 'FollowUp': 'null', 'Medicines': [{'name': 'Stamlo 5Mg Tablets', 'quantity': '8', 'route': 'oral', 'instruction': 'null', 'duration': '4 days', 'frequency': '0-1-1'}, {'name': 'ZECAL 500MG TABLETS', 'quantity': '24', 'route': 'oral', 'instruction': 'null', 'duration': '12 days', 'frequency': '0-1-1'}]}


In [11]:
# # Split the dataset into 80% training and 20% validation
# split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = split_dataset["train"]
# val_dataset = split_dataset["test"]
        ## OR
train_val_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
val_test_split = train_val_split["test"].train_test_split(test_size=0.2, seed=42)

train_dataset = train_val_split["train"]
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

In [12]:
print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(8000, 5)
(1600, 5)
(400, 5)


In [13]:
# !pip install evaluate

In [14]:
# from datasets import load_metric
# import numpy as np

# # Example metric (e.g., accuracy for classification tasks)
# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)
import evaluate
import numpy as np

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # return metric.compute(predictions=predictions, references=labels)
    return {"accuracy": metric.compute(predictions=predictions, references=labels)}



In [15]:
# # Load the fine-tuned model and tokenizer
# tokenizer = T5Tokenizer.from_pretrained('./t5-finetuned-tokenizer')
# model = T5ForConditionalGeneration.from_pretrained('./t5-finetuned-model').to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-5,                       # Learning rate
    per_device_train_batch_size=8,            # Batch size for training
    per_device_eval_batch_size=8,             # Batch size for evaluation
    num_train_epochs=12,                       # Number of epochs
    weight_decay=0.01,                        # Weight decay
    warmup_steps=300,                         # Warmup steps
    gradient_accumulation_steps=4,            # Gradient accumulation
    optim="adamw_torch",                      # Optimizer (AdamW)
    save_strategy="epoch",                    # Save every epoch
    logging_dir="./logs",                     # Log directory
    overwrite_output_dir=True,
    eval_steps= 200, ## 400,
    save_steps= 200, ## 500,
    logging_steps= 100,
    load_best_model_at_end=True,  # Load best model at the end based on evaluation
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    save_total_limit=3,                    # Limit saved models
    eval_accumulation_steps=4,             # Accumulate eval gradients
)


optimizer = AdamW(model.parameters(), lr= 0.0002,
                weight_decay= 0.01)

num_training_steps = len(train_dataset) * training_args.num_train_epochs // training_args.per_device_train_batch_size

scheduler = get_scheduler("linear", optimizer=optimizer, 
                            num_warmup_steps=300, num_training_steps=num_training_steps)

# Trainer for fine-tuning the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator= data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    # compute_metrics=compute_metrics
    )

# Resume training
trainer.train()

  3%|▎         | 100/3000 [18:58<9:02:26, 11.22s/it]

{'loss': 4.6037, 'grad_norm': 0.6187983155250549, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.4}


  7%|▋         | 200/3000 [37:41<8:47:22, 11.30s/it]

{'loss': 0.4192, 'grad_norm': 0.30518123507499695, 'learning_rate': 0.00013333333333333334, 'epoch': 0.8}


                                                    
  8%|▊         | 250/3000 [47:46<8:34:58, 11.24s/it]

...

{'eval_loss': 0.0038227690383791924, 'eval_runtime': 43.1597, 'eval_samples_per_second': 37.072, 'eval_steps_per_second': 4.634, 'epoch': 12.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 3000/3000 [11:05:15<00:00, 13.57s/it]

{'train_runtime': 39916.0227, 'train_samples_per_second': 2.405, 'train_steps_per_second': 0.075, 'train_loss': 0.1919476176102956, 'epoch': 12.0}


100%|██████████| 3000/3000 [11:05:16<00:00, 13.31s/it]


TrainOutput(global_step=3000, training_loss=0.1919476176102956, metrics={'train_runtime': 39916.0227, 'train_samples_per_second': 2.405, 'train_steps_per_second': 0.075, 'total_flos': 6496406470656000.0, 'train_loss': 0.1919476176102956, 'epoch': 12.0})

In [16]:
trainer.evaluate()

100%|██████████| 200/200 [00:41<00:00,  4.79it/s]


{'eval_loss': 0.0038227690383791924,
 'eval_runtime': 41.7085,
 'eval_samples_per_second': 38.361,
 'eval_steps_per_second': 4.795,
 'epoch': 12.0}

In [17]:
# Save the model again if needed
model.save_pretrained('./t5-segregate-model-21dec')
tokenizer.save_pretrained('./t5-segregate-tokenizer-21dec')

('./t5-segregate-tokenizer-21dec\\tokenizer_config.json',
 './t5-segregate-tokenizer-21dec\\special_tokens_map.json',
 './t5-segregate-tokenizer-21dec\\spiece.model',
 './t5-segregate-tokenizer-21dec\\added_tokens.json',
 './t5-segregate-tokenizer-21dec\\tokenizer.json')

In [18]:
test_results = trainer.evaluate(test_dataset)
print(f"Test Results: {test_results}")

100%|██████████| 50/50 [00:10<00:00,  4.74it/s]

Test Results: {'eval_loss': 0.004462759010493755, 'eval_runtime': 10.5828, 'eval_samples_per_second': 37.797, 'eval_steps_per_second': 4.725, 'epoch': 12.0}


In [19]:
# predictions = trainer.predict(test_dataset)

# # Access logits, predictions, and label IDs
# logits = predictions.predictions
# labels = predictions.label_ids

# # Compute any custom metrics
# print(predictions.metrics)


In [20]:

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# tokenizer = AutoTokenizer.from_pretrained('./t5-segregate-tokenizer-05dec-2')
# model = AutoModelForSeq2SeqLM.from_pretrained('./t5-segregate-model-05dec-2').to(device)
tokenizer = AutoTokenizer.from_pretrained('./t5-segregate-tokenizer-21dec')
model = AutoModelForSeq2SeqLM.from_pretrained('./t5-segregate-model-21dec').to(device)


def generate_output(paragraph):
    paragraph = paragraph.replace("'", "")
    inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, padding='max_length', max_length=256).to(device)
    # print(inputs)
    outputs = model.generate(**inputs, max_length=512, num_beams=2, early_stopping=True)
    # print(outputs)
    # print(outputs[0])
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


# result = generate_output("The patient presents with a chief complaint of Skin rash may be this disease comes from friends or family. Symptoms reported include Feeling of nausea, with occasional vomiting. After discussing patient medical history and current symptoms, it seems likely that patient is experiencing this problem a long ago. Ultrasound of the abdomen, Blood test for amylase, Differential leucocyte count (dlc),   these investigation test you have to do. For treatment, Prednisolone 10 mg, Topiramate 50 mg were given. The patient will follow up in a week for reassessment.")
# result = generate_output("The patient presents with a history of intermittent palpitations and dizziness. A 24-hour Holter monitor is recommended to assess for any arrhythmias. Initial management includes a low-dose beta-blocker to control symptoms.")
# result = generate_output("The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional dizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar episodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), chest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to rule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin sublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was scheduled for further evaluation based on investigation results.")
# result = generate_output("A 30-year-old female presents with fatigue, muscle aches, and low-grade fever for the past five days. She reports a recent history of upper respiratory infection two weeks prior, which has since resolved. On examination, her throat is mildly inflamed, and her lymph nodes are slightly enlarged. No signs of pneumonia on chest auscultation. Given the symptoms, I suspect she may be experiencing a viral syndrome or post-viral fatigue. I recommended ibuprofen for muscle pain, hydration, and rest. To rule out mononucleosis or thyroid dysfunction, I've ordered a complete blood count (CBC) and thyroid function tests. She should return if symptoms persist beyond two weeks.")
# result = generate_output("The patient, a 45-year-old male, presents with a chief complaint of intermittent chest pain for the past week. He describes the pain as a pressure-like sensation, radiating to the left arm, particularly during physical exertion. The patient has a history of hypertension, which he is managing with amlodipine. No other major comorbidities are reported. Upon examination, his vitals are stable, but an ECG reveals minor ST segment changes. My initial diagnosis is suspected angina. I've prescribed nitroglycerin sublingually for acute episodes and referred him for a stress test and lipid profile to further assess his cardiovascular risk. Follow-up in one week.")

# result = generate_output("Reported signs of stomach hurts, weight gain led me to order ct angiography, Absolute eosinophil count(aec), GAD - 65. The patient was prescribed medications including Metolar Injection, and PROLOMET XL 25MG TABLETS.")
# result = generate_output("A patient complains of a runny nose and sore throat for 3 days. Symptoms include sneezing, nasal congestion, mild fever, and cough. No investigations are required unless symptoms persist; a throat swab can be done to rule out bacterial infection if necessary. Treatment includes paracetamol for fever, antihistamines for congestion, and a nasal decongestant spray for relief.")
# result = generate_output("The patient presents with high-grade fever lasting 3 days. Associated symptoms include chills, body ache, and headache. Investigations such as a complete blood count (CBC), malaria antigen, typhoid, or dengue tests are advised based on local endemic conditions. Paracetamol 500mg every 6 hours for fever, hydration, and rest are recommended. Antibiotics may be considered if bacterial infection is confirmed.")
# result = generate_output("Patient is having hand pain & elbow pain. I have told him to take neospring 100 tabs. I have told him to check cbc and blood pressure.")
# result = generate_output("The patient is suffering from mild fever, body aches, and general weakness, which could be due to a common viral infection. To manage the symptoms, Dolo 650 is prescribed, one tablet every six hours as needed for fever, and Paracetamol, one tablet every six hours for pain relief. A Complete Blood Count (CBC) and Liver Function Test (LFT) are suggested to rule out any underlying conditions.")


# result = generate_output("The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional dizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar episodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), chest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to rule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin sublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was scheduled for further evaluation based on investigation results.")
# result = generate_output("A patient complains of a runny nose and sore throat for 3 days. Symptoms include sneezing, nasal congestion, mild fever, and cough. No investigations are required unless symptoms persist; a throat swab can be done to rule out bacterial infection if necessary. Treatment includes paracetamol for fever, antihistamines for congestion, and a nasal decongestant spray for relief.")
result = generate_output("The patient complains of a burning sensation in the chest, particularly after meals. Symptoms include heartburn, belching, and a sour taste in the mouth. Initial investigations may not be necessary, but endoscopy is recommended if symptoms persist or worsen. Omeprazole 20mg once daily before meals and antacids as needed are prescribed.")




# print(type(result))
print(result)

result = result.replace("'",'"').replace("<", "{").replace(">", "}")
import json
print(json.dumps(result, indent=4))

'ChiefComplaint': 'The patient is experiencing heartburn, belching, and a sour taste in the mouth.', 'Symptoms': ['heartburn', 'belching', 'a sour taste in the mouth'], 'LabTest': ['null', 'endoscopy'], 'Advice': [], 'FollowUp': 'null', 'Medicines': ['name': 'Omeprazole 20mg', 'quantity': 'null', 'route': 'null', 'instruction': 'before meals', 'duration': 'null', 'frequency': '1-0-0', 'name': 'antacids as needed are prescribed.', 'quantity': 'null', 'route': 'null', 'instruction': 'null', 'duration': 'null', 'frequency': 'null']
"\"ChiefComplaint\": \"The patient is experiencing heartburn, belching, and a sour taste in the mouth.\", \"Symptoms\": [\"heartburn\", \"belching\", \"a sour taste in the mouth\"], \"LabTest\": [\"null\", \"endoscopy\"], \"Advice\": [], \"FollowUp\": \"null\", \"Medicines\": [\"name\": \"Omeprazole 20mg\", \"quantity\": \"null\", \"route\": \"null\", \"instruction\": \"before meals\", \"duration\": \"null\", \"frequency\": \"1-0-0\", \"name\": \"antacids as ne

In [32]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = AutoTokenizer.from_pretrained('./t5-segregate-tokenizer-16dec')
# model = AutoModelForSeq2SeqLM.from_pretrained('./t5-segregate-model-16dec').to(device)
# tokenizer = AutoTokenizer.from_pretrained('./t5-segregate-tokenizer-19dec')
# model = AutoModelForSeq2SeqLM.from_pretrained('./t5-segregate-model-19dec').to(device)
# tokenizer = AutoTokenizer.from_pretrained('./t5-segregate-tokenizer-20dec')
# model = AutoModelForSeq2SeqLM.from_pretrained('./t5-segregate-model-20dec').to(device)


def generate_output(paragraph):
    print(paragraph)
    paragraph = paragraph.replace("'", "")
    inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, padding='max_length', max_length=256).to(device)
    outputs = model.generate(**inputs, max_length=512, num_beams=2, early_stopping=True)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


# result = generate_output("The patient presents with low back pain lasting a week, worsened by sitting or bending. Symptoms include stiffness and radiating pain to the leg. Investigations such as an X-ray are recommended if trauma is involved, while an MRI may be needed for suspected radiculopathy. Treatment involves NSAIDs, muscle relaxants, and physiotherapy exercises.")
# result = generate_output("The patient is suffering from severe headache, nausea, and sensitivity to light. A brain MRI and blood sugar test have been recommended. The patient is prescribed ibuprofen 400 mg, 15 tablets, to be taken with food twice daily, and anti-nausea tablets (ondansetron), 10 tablets, as needed. He is advised to avoid bright lights, stay hydrated, and follow a regular sleep schedule. The next visit is planned in 10 days.")

# result = generate_output("The patient complains of abdominal pain, bloating, and constipation for the past week. He is advised to undergo an abdominal ultrasound, stool test, and thyroid function test. He has been prescribed a laxative syrup, 10 ml at bedtime, and antacid tablets, 15 tablets, to be taken before meals. He is advised to eat fiber-rich foods, drink warm water, and avoid spicy meals. A follow-up appointment is set for two weeks later.")
# result = generate_output("The patient reports cold, throat irritation, and mild fever for three days.The patient is prescribed cetirizine 10 mg, 10 tablets, to be taken at night, and paracetamol 500 mg, 15 tablets, as needed for fever after food day and night. A throat swab and CBC have been recommended. He is advised to drink warm water, consume soups, and rest adequately. The next visit is scheduled in one week.")
# result = generate_output("The patient presents with persistent coughing, chest congestion, and difficulty breathing. A chest X-ray and sputum test are recommended. The patient is prescribed an expectorant syrup, 2 teaspoons three times daily, and an antibiotic course of amoxicillin 500 mg, 15 capsules, to be taken after meals. He is advised to avoid cold drinks, inhale steam twice daily, and stay hydrated. A follow-up is planned in two weeks.")
# result = generate_output("A routine consultation for hypertension management. The patient mentions occasional dizziness and headaches. Blood pressure monitoring, kidney function tests, and a lipid profile are advised. Medication adjustments might be needed; ensure adherence to current antihypertensives and reinforce lifestyle changes like a low-sodium diet.")
# result = generate_output("The patient, a 60-year-old male, presented with complaints of swelling in his legs, fatigue, and shortness of breath over the past three weeks. He has a history of hypertension, controlled with medication, and is a former smoker with a 20-year smoking history. His physical examination revealed pitting edema in both legs and crackles upon auscultation of the lungs. His blood pressure was elevated at 160/100 mmHg, and his pulse was irregular at 110 bpm. Given the possibility of heart failure or pulmonary hypertension, I have ordered a chest X-ray, echocardiogram, and BNP blood test to assess heart function. I have also recommended adjusting his current antihypertensive medication and adding a diuretic to manage the fluid retention. The patient has been advised to restrict salt intake and monitor his weight daily to track fluid accumulation. He has been scheduled for a follow-up in one week to review test results and assess his response to the treatment. If his condition does not improve, we may consider referring him to a cardiologist for further evaluation.")
# result = generate_output("Ram is having some problems cold & stomach pain & head pain, told him to take dolo 650mg after food daily morning for 10 days, paracetamol daily mornig, nigth and noon, olmessar 10 tablets. A cbc and lft are suggested to do. Properly hydrate your body regularly. Your next checkup date will be on december 30.")
# result = generate_output("A patient complains of a runny nose and sore throat for 3 days. Symptoms include sneezing, nasal congestion, mild fever, and cough. No investigations are required unless symptoms persist; a throat swab can be done to rule out bacterial infection if necessary. Treatment includes paracetamol for fever, antihistamines for congestion, and a nasal decongestant spray for relief.")
# result = generate_output("The patient complains of a burning sensation in the chest, particularly after meals. Symptoms include heartburn, belching, and a sour taste in the mouth. Initial investigations may not be necessary, but endoscopy is recommended if symptoms persist or worsen. Properly hydrate your body regularly. And don't eat any junk food. Your next checkup date will be on december 30. Omeprazole 20mg once daily before meals and antacids as needed are prescribed.")
result = generate_output("The patient, a 35-year-old male, presented with complaints of persistent abdominal pain, nausea, and occasional diarrhea for the past two weeks. Suspecting gastritis, I have advised a complete blood count, stool test, and an abdominal ultrasound to rule out underlying conditions. The patient has been prescribed a proton pump inhibitor for symptom relief and advised to avoid spicy foods and caffeine and alcohol, along with maintaining a bland diet. A follow-up appointment is scheduled for two weeks to review test results and monitor progress.")
# result = generate_output("The patient, a 60-year-old female, reported frequent urination, increased thirst, and fatigue over the last month, raising suspicion of diabetes. A fasting blood sugar, HbA1c, and urine test were recommended, and the patient was prescribed metformin to manage blood sugar levels. Lifestyle changes, including a low-carb diet and daily exercise, were advised, with a follow-up in two weeks.")


print(result)
print()
import re
import json

def parse_input_to_json(input_text):
    # Replace single quotes with double quotes
    input_text = re.sub(r"'([^']*)'", r'"\1"', input_text)

    # Extract medicine content
    # medicine_match = re.search(r'"medicine": \[(.*?)\]', input_text)
    medicine_match = re.search(r'"Medicines": \[(.*?)\]', input_text)
    if medicine_match:
        medicine_content = medicine_match.group(1)

        # Initialize variables
        medicine_list = []
        current_dict = {}
        keys = ["name", "quantity", "route", "instruction", "duration", "frequency"]

        # Find all key-value pairs
        key_value_pairs = re.findall(r'("name"|"quantity"|"route"|"instruction"|"duration"|"frequency"): ("[^"]*"|null)', medicine_content)
        for key, value in key_value_pairs:
            # Clean key and value
            # print(value)
            key = key.strip('"')
            value = value.strip('"') if value != "null" else None

            # Add to the current dictionary
            if key in current_dict:
                # If the key already exists, start a new dictionary
                medicine_list.append(current_dict)
                current_dict = {}

            current_dict[key] = value

            # If all keys are present, save the dictionary and start a new one
            if len(current_dict) == len(keys):
                medicine_list.append(current_dict)
                current_dict = {}

        # Add the last dictionary if not empty
        if current_dict:
            medicine_list.append(current_dict)

        # Replace the medicine content in the original input
        input_text = re.sub(r'"Medicines": \[.*?\]', f'"Medicines": {json.dumps(medicine_list)}', input_text)
        # input_text = re.sub(r'"medicine": \[.*?\]', f'"medicine": {json.dumps(medicine_list)}', input_text)
        # print(input_text[len(input_text) - 3:])
        input_text.replace(": nan", ': null')

    # Convert the entire input into JSON
    try:
        structured_data = json.loads("{" + input_text + "}")
        return structured_data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None

structured_data = parse_input_to_json(result)
if structured_data:
    print(json.dumps(structured_data, indent=4))


The patient, a 35-year-old male, presented with complaints of persistent abdominal pain, nausea, and occasional diarrhea for the past two weeks. Suspecting gastritis, I have advised a complete blood count, stool test, and an abdominal ultrasound to rule out underlying conditions. The patient has been prescribed a proton pump inhibitor for symptom relief and advised to avoid spicy foods and caffeine and alcohol, along with maintaining a bland diet. A follow-up appointment is scheduled for two weeks to review test results and monitor progress.
'ChiefComplaint': 'The patient is experiencing persistent abdominal pain, nausea, and occasional diarrhea.', 'Symptoms': ['continuous abdominal pain', 'nausea', 'occasional diarrhea'], 'LabTest': ['gastritis'], 'Advice': ['a complete blood count','stools test', 'abdominal ultrasound'], 'FollowUp': 'A follow-up appointment is scheduled for two weeks to review test results and monitor progress.', 'Medicines': ['name': 'pron pump inhibitor for symptom

In [22]:


def parse_input_to_json(input_text):
    print("Original Input:")
    print(input_text)  # Debug input text

    # Step 1: Replace single quotes with double quotes
    input_text = re.sub(r"'([^']*)'", r'"\1"', input_text)
    print("\nAfter Replacing Single Quotes:")
    print(input_text)  # Debug step 1

    # Step 2: Handle Medicines list

    medicines_match = re.search(r'"Medicines": \[(.*?)\]', input_text, re.DOTALL)
    # medicines_match = re.search(r'"medicine": \[(.*?)\]', input_text, re.DOTALL)
    if medicines_match:
        print("\nExtracted Medicines Content:")
        print(medicines_match.group(1))  # Debug extracted Medicines section
        
        medicine_content = medicines_match.group(1)

        # Extract key-value pairs
        # Use regex to extract key-value pairs for Medicines
        key_value_pairs = re.findall(r'"(name|quantity|route|instruction|duration|frequency)":\s*"([^"]*|null)"', medicine_content)
        print("\nKey-Value Pairs in Medicines:")
        print(key_value_pairs)  # Debug key-value pairs

        # Build list of dictionaries for Medicines
        medicines_list = []
        current_medicine = {}
        keys = ["name", "quantity", "route", "instruction", "duration", "frequency"]

        for key, value in key_value_pairs:
            current_medicine[key] = value if value != "null" else None

            # When all keys are captured, append to the list and reset
            if len(current_medicine) == len(keys):
                medicines_list.append(current_medicine)
                current_medicine = {}

        print("\nParsed Medicines List:")
        print(medicines_list)  # Debug final list

        # Replace the Medicines part with the fixed JSON list
        
        input_text = re.sub(r'"Medicines": \[.*?\]', f'"Medicines": {json.dumps(medicines_list)}', input_text, flags=re.DOTALL)
        # input_text = re.sub(r'"medicine": \[.*?\]', f'"medicine": {json.dumps(medicines_list)}', input_text, flags=re.DOTALL)
        print("\nAfter Replacing Medicines Section:")
        print(input_text)  # Debug updated text

    # Step 3: Parse the JSON
    try:
        structured_data = json.loads("{" + input_text + "}")
        return structured_data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None


structured_data = parse_input_to_json(result)
if structured_data:
    print(json.dumps(structured_data, indent=4))

Original Input:
'ChiefComplaint': 'The patient is experiencing cold, throat irritation, and mild fever for three days.', 'Symptoms': ['cold', 'throat irritation','mild fever'], 'LabTest': ['a throat swab', 'CBC'], 'Advice': ['Drink warm water, consume soups, and rest adequately.'], 'FollowUp': 'The next visit is scheduled in one week.', 'Medicines': ['name': 'cetizine 10 mg', 'quantity': '10', 'route': 'null', 'instruction': 'null', 'duration': 'null', 'frequency': '0-0-1', 'name': 'paracetamol 500 mg', 'quantity': '15', 'route': 'null', 'instruction': 'after food', 'duration': 'null', 'frequency': '1-0-1']

After Replacing Single Quotes:
"ChiefComplaint": "The patient is experiencing cold, throat irritation, and mild fever for three days.", "Symptoms": ["cold", "throat irritation","mild fever"], "LabTest": ["a throat swab", "CBC"], "Advice": ["Drink warm water, consume soups, and rest adequately."], "FollowUp": "The next visit is scheduled in one week.", "Medicines": ["name": "cetizin

Input ==>
"The patient complains of a burning sensation in the chest, particularly after meals. Symptoms include heartburn, belching, and a sour taste in the mouth. Initial investigations may not be necessary, but endoscopy is recommended if symptoms persist or worsen. Omeprazole 20mg once daily before meals and antacids as needed are prescribed."


Output ==>
"chiefComplaint": "The patient is experiencing a burning sensation in the chest, and heartburn, belching, and a sour taste in the mouth.", "symptoms": ["burning sensation in the chest", "belching", "a sour taste in the mouth"], 
"tests": ["endoscopy"], 
"medicine": ["name": "omeprazole 20mg", "quantity": "null", "route": "null", "instruction": "before meals", "duration": "1 days", "frequency": "1-0-0"]



Input ==>
"The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional dizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar episodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), chest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to rule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin sublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was scheduled for further evaluation based on investigation results."


output ==>D
"chiefComplaint": "The patient is experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, and shortness of breath.", 
"symptoms": ["intermittent, sharp chest pain on the left side", "palpitations", "occasional dizziness"],
"tests": ["electrocardiogram (ECG)", "chest X-ray", "complete blood count (CBC)"], 
"medicine": ["name": "aspirin 81 mg daily", "quantity": "20", "route": "oral", "instruction": "null", "duration": "20 days", "frequency": "1-0-0"]


Input ==>
"The patient presents with low back pain lasting a week, worsened by sitting or bending. Symptoms include stiffness and radiating pain to the leg. Investigations such as an X-ray are recommended if trauma is involved, while an MRI may be needed for suspected radiculopathy. Treatment involves NSAIDs, muscle relaxants, and physiotherapy exercises."


output ==>
"chiefComplaint": "The patient is experiencing low back pain, worsened by sitting or bending, and shaky.", 
"symptoms": ["low back pain", "stiffness", "radiating pain"], 
"tests": ["an X-ray"], 
"medicine": ["name": "NSAIDs", "quantity": "null", "route": "null", "instruction": "null", "duration": "1 days", "frequency": "null"]

In [23]:
# dm_js = json.dumps(result)
# dm_js = dm_js.replace('\\"', '"')
# dm_js = dm_js.replace("'", '"')
# dm_js

In [24]:
# print(len(tokenizer.vocab.keys()))

print(tokenizer.tokenize("<START_JSON> Hi, I am a {Robot} <END_JSON>"))

['▁', '<', 'STAR', 'T', '_', 'J', 'SON', '>', '▁Hi', ',', '▁I', '▁am', '▁', 'a', '▁', '{', 'Rob', 'o', 't', '}', '▁', '<', 'END', '_', 'J', 'SON', '>']


In [25]:
"""

The patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. 
The pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional 
dizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar 
episodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), 
chest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to 
rule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin 
sublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was 
scheduled for further evaluation based on investigation results.


"""

'\n\nThe patient reports experiencing intermittent, sharp chest pain on the left side, radiating to the left arm, over the past month. \nThe pain is aggravated by exertion and relieved with rest. Accompanying symptoms include shortness of breath, palpitations, and occasional \ndizziness. The patient has a history of hypertension, managed with lisinopril, and a 20-pack-year smoking history. No history of similar \nepisodes, nausea, or syncope. Given the clinical presentation, the patient was advised to undergo an electrocardiogram (ECG), \nchest X-ray, and complete blood count (CBC) to evaluate potential cardiac and pulmonary causes. A troponin test was also recommended to \nrule out acute coronary syndrome. The patient was started on aspirin 81 mg daily and advised to continue lisinopril. Nitroglycerin \nsublingual tablets were prescribed for acute chest pain relief. Smoking cessation counseling was provided, and a follow-up appointment was \nscheduled for further evaluation based on i

In [26]:
# !pip install optimum
# !pip install onnx onnxruntime
# !pip install optimum[onnxruntime]


In [27]:
# import onnxruntime as ort
# from transformers import AutoTokenizer
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import numpy as np
# from optimum.onnxruntime import ORTModelForSeq2SeqLM


# # model = ORTModelForSeq2SeqLM.from_pretrained("./t5-segregate-model-20sept", from_transformers=True)
# model = ORTModelForSeq2SeqLM.from_pretrained("./t5-segregate-model-20sept", export=True)
# # model.save_pretrained("optimized_model")
# model.save_pretrained("./tmp/onnx/")
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=r"C:\Users\jites\Desktop\Project_folder\Jupyter_practice\jupyter\t5-segregate-tokenizer-2oct")
# # tokenizer = T5Tokenizer.from_pretrained('./t5-segregate-tokenizer-20sept')

# # Load ONNX runtime sessions
# encoder_session = ort.InferenceSession("./tmp/onnx/encoder_model.onnx")
# decoder_session = ort.InferenceSession("./tmp/onnx/decoder_model.onnx")
# decoder_with_past_session = ort.InferenceSession("./tmp/onnx/decoder_with_past_model.onnx")

# # input_text = "Translate English to French: How are you?"
# input_text = "The patient, a 45-year-old male, presents with a chief complaint of intermittent chest pain for the past week. He describes the pain as a pressure-like sensation, radiating to the left arm, particularly during physical exertion. The patient has a history of hypertension, which he is managing with amlodipine. No other major comorbidities are reported. Upon examination, his vitals are stable, but an ECG reveals minor ST segment changes. My initial diagnosis is suspected angina. I've prescribed nitroglycerin sublingually for acute episodes and referred him for a stress test and lipid profile to further assess his cardiovascular risk. Follow-up in one week."

# # Tokenize the input
# inputs = tokenizer(input_text, return_tensors="np", padding=True)

# # Convert inputs to int64
# input_ids = inputs["input_ids"].astype(np.int64)
# attention_mask = inputs["attention_mask"].astype(np.int64)

# # Encoder inference
# encoder_outputs = encoder_session.run(
#     None, {"input_ids": input_ids, "attention_mask": attention_mask}
# )

# # Prepare decoder inputs
# decoder_input_ids = tokenizer("<pad>", return_tensors="np")["input_ids"]

# # Decoder inference
# decoder_outputs = decoder_session.run(
#     None,
#     {
#         "input_ids": decoder_input_ids.astype(np.int64),
#         "encoder_hidden_states": encoder_outputs[0],
#         "encoder_attention_mask": attention_mask,
#     },
# )

# # output_text = tokenizer.decode(decoder_outputs[0][0], skip_special_tokens=True)
# # print(output_text)

# decoded_token_lst_ids = np.argmax(decoder_outputs[0], axis=-1)
# decoded_token_lst_ids = decoded_token_lst_ids[0].tolist()
# # decoded_token_lst_ids = decoder_outputs[0].tolist()[0]
# output_text = tokenizer.decode(decoded_token_lst_ids, skip_special_tokens=True)
# print(output_text)